In [4]:
# Import required libraries
import rasterio
import numpy as np
from rasterio.merge import merge
from pathlib import Path

# Set up display for notebook
%matplotlib inline
import matplotlib.pyplot as plt

# Function to read raster metadata
def get_raster_info(raster_path):
    """
    Extract key metadata from a raster file
    """
    with rasterio.open(raster_path) as src:
        return {
            'crs': src.crs,
            'transform': src.transform,
            'nodata': src.nodata,
            'count': src.count,
            'width': src.width,
            'height': src.height
        }

# Function to perform MAX overlay
def overlay_rasters_max(raster_paths):
    """
    Overlay multiple rasters using MAX criteria
    Maintains original CRS and NODATA values
    """
    # Read first raster to get reference metadata
    with rasterio.open(raster_paths[0]) as src:
        ref_meta = src.meta.copy()
        ref_nodata = src.nodata
        
        # Initialize output array with first raster
        output = src.read(1)
        
        # Handle nodata values
        if ref_nodata is not None:
            output = np.where(output == ref_nodata, np.nan, output)
    
    # Process remaining rasters
    for path in raster_paths[1:]:
        with rasterio.open(path) as src:
            data = src.read(1)
            
            # Handle nodata values
            if src.nodata is not None:
                data = np.where(data == src.nodata, np.nan, data)
            
            # Perform MAX operation
            output = np.nanmax([output, data], axis=0)
    
    # Replace NaN with original nodata value
    if ref_nodata is not None:
        output = np.where(np.isnan(output), ref_nodata, output)
    
    return output, ref_meta

# Example usage
# Replace these paths with your actual raster file paths
raster_paths = [
    'C:/Users/matti/Downloads/STORM_FIXED_RETURN_PERIODS_PRESENT_TIF/STORM_FIXED_RETURN_PERIODS_NA_100_YR_RP.tif',
    'C:/Users/matti/Downloads/STORM_FIXED_RETURN_PERIODS_PRESENT_TIF/STORM_FIXED_RETURN_PERIODS_NI_100_YR_RP.tif',
    'C:/Users/matti/Downloads/STORM_FIXED_RETURN_PERIODS_PRESENT_TIF/STORM_FIXED_RETURN_PERIODS_SI_100_YR_RP.tif',
    'C:/Users/matti/Downloads/STORM_FIXED_RETURN_PERIODS_PRESENT_TIF/STORM_FIXED_RETURN_PERIODS_SP_100_YR_RP.tif',
    'C:/Users/matti/Downloads/STORM_FIXED_RETURN_PERIODS_PRESENT_TIF/STORM_FIXED_RETURN_PERIODS_EP_100_YR_RP.tif',
    'C:/Users/matti/Downloads/STORM_FIXED_RETURN_PERIODS_PRESENT_TIF/STORM_FIXED_RETURN_PERIODS_WP_100_YR_RP.tif',
    'X:/Work/WB/Risk_tools/script/data/HZD/GLB/STORM/test.tif'
]

# Display input raster information
print("Input Raster Information:")
for path in raster_paths:
    print(f"\nRaster: {Path(path).name}")
    info = get_raster_info(path)
    for key, value in info.items():
        print(f"{key}: {value}")

# Perform overlay operation
result, metadata = overlay_rasters_max(raster_paths)

# Save the result
output_path = 'X:/Work/WB/Risk_tools/script/data/HZD/GLB/STORM/rp100.tif'

# Update metadata for output
metadata.update({
    'driver': 'GTiff',
    'height': result.shape[0],
    'width': result.shape[1],
    'count': 1
})

# Write output raster
with rasterio.open(output_path, 'w', **metadata) as dst:
    dst.write(result, 1)

# Visualize results
plt.figure(figsize=(12, 8))
plt.imshow(result)
plt.colorbar(label='Value')
plt.title('MAX Overlay Result')
plt.show()

# Display output information
print("\nOutput Raster Information:")
output_info = get_raster_info(output_path)
for key, value in output_info.items():
    print(f"{key}: {value}")

Input Raster Information:

Raster: STORM_FIXED_RETURN_PERIODS_NA_100_YR_RP.tif
crs: EPSG:4326
transform: | 0.10, 0.00,-104.95|
| 0.00,-0.10, 59.95|
| 0.00, 0.00, 1.00|
nodata: None
count: 1
width: 1040
height: 550

Raster: STORM_FIXED_RETURN_PERIODS_NI_100_YR_RP.tif
crs: EPSG:4326
transform: | 0.10, 0.00, 30.05|
| 0.00,-0.10, 59.95|
| 0.00, 0.00, 1.00|
nodata: None
count: 1
width: 700
height: 550

Raster: STORM_FIXED_RETURN_PERIODS_SI_100_YR_RP.tif
crs: EPSG:4326
transform: | 0.10, 0.00, 10.05|
| 0.00,-0.10,-5.15|
| 0.00, 0.00, 1.00|
nodata: None
count: 1
width: 1250
height: 550

Raster: STORM_FIXED_RETURN_PERIODS_SP_100_YR_RP.tif
crs: EPSG:4326
transform: | 0.10, 0.00, 135.05|
| 0.00,-0.10,-5.15|
| 0.00, 0.00, 1.00|
nodata: None
count: 1
width: 1050
height: 550

Raster: STORM_FIXED_RETURN_PERIODS_EP_100_YR_RP.tif
crs: EPSG:4326
transform: | 0.10, 0.00,-179.95|
| 0.00,-0.10, 59.95|
| 0.00, 0.00, 1.00|
nodata: None
count: 1
width: 1050
height: 550

Raster: STORM_FIXED_RETURN_PERIODS_WP_

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (2, 550) + inhomogeneous part.